In [ ]:
# data : https://www.smard.de/home/downloadcenter/download-marktdaten#!?downloadAttributes=%7B%22selectedCategory%22:3,%22selectedSubCategory%22:8,%22selectedRegion%22:%22DE%22,%22from%22:1658613600000,%22to%22:1659563999999,%22selectedFileType%22:%22CSV%22%7D

In [ ]:
import pandas as pd
import pickle
from pathlib import Path
import numpy as np
import sklearn.preprocessing 
import matplotlib.pyplot as plt

In [ ]:
df_smard = pd.read_csv('../data/energy_data/elect_price_2022.csv', delimiter=';', decimal=',', parse_dates=[['Datum', 'Uhrzeit']])
df_smard.tail()

In [ ]:
ser_energy_price_germany = df_smard[['Datum_Uhrzeit', 'Deutschland/Luxemburg[€/MWh]']]
print(f"Type: {type(ser_energy_price_germany)}")
ser_energy_price_germany.head()

In [ ]:
type(ser_energy_price_germany['Datum_Uhrzeit'])

In [ ]:
mask = ser_energy_price_germany['Datum_Uhrzeit'].duplicated()

print(ser_energy_price_germany[mask])

In [ ]:
ser_energy_price_germany = ser_energy_price_germany.drop_duplicates(subset=['Datum_Uhrzeit'])

In [ ]:
idx = ser_energy_price_germany['Datum_Uhrzeit']
val = ser_energy_price_germany['Deutschland/Luxemburg[€/MWh]']
ser = val.set_axis(idx, axis=0)

In [ ]:
ser.head()

In [ ]:
ser.tail()

In [ ]:
ser_sept = ser.loc["2022-09-01 00:00:00":"2022-09-30 23:00:00"]
ser_oct = ser.loc["2022-10-01 00:00:00":"2022-10-31 23:00:00"]

In [ ]:
ser_sept.tail()

In [ ]:
ser_oct.tail()

In [ ]:
# Preprocessing: Using MinMaxScaler
# Since the data is static it might be better to do it once than before every training



In [ ]:
#upsamled_ser = ser.resample('60s').pad()
#TODO: to datetime?? search for resample
upsamled_ser_sept = ser_sept.resample('60s').pad()
upsamled_ser_oct = ser_oct.resample('60s').pad()

In [ ]:
upsamled_ser_sept.tail()

In [ ]:
upsamled_ser_oct.tail()

In [ ]:
# Euro/ MWh to Euro/ kWmin
rescaled_ser_sept = upsamled_ser_sept.mul(1/(1000*60))
rescaled_ser_oct = upsamled_ser_oct.mul(1/(1000*60))

In [ ]:
rescaled_ser_sept.tail()

In [ ]:
rescaled_ser_oct.tail()

In [ ]:
energy_price_lst_sept = rescaled_ser_sept.to_list()

with open('../data/energy_data/elect_price_sept_kwmin.pkl', 'wb') as outfile:
    pickle.dump(energy_price_lst_sept, outfile)

In [ ]:
np_arr = rescaled_ser_oct.to_numpy()

In [ ]:
print(np_arr)

In [ ]:
energy_price_lst_oct = rescaled_ser_oct.to_list()

#with open('../data/energy_data/elect_price_oct_kwmin.pkl', 'wb') as outfile:
    #pickle.dump(energy_price_lst_sept, outfile)

In [ ]:
energy_price_lst_sept_oct = energy_price_lst_sept.append(energy_price_lst_oct)
#with open('../data/energy_data/elect_price_sept_oct_kwmin.pkl', 'wb') as outfile:
    #pickle.dump(energy_price_lst_sept_oct, outfile)

In [ ]:
### Generate 14 days ahead data
### MinMax and true values

ser_sept_14_days = ser_sept.loc["2022-09-01 00:00:00":"2022-09-14 23:00:00"]

ser_sept_14_days.plot()

In [ ]:

print(f"min: {ser_sept_14_days.min()}")
print(f"max: {ser_sept_14_days.max()}")


In [ ]:
ser_sept_14_days = ser_sept.loc["2022-09-01 00:00:00":"2022-09-14 23:00:00"]
upsampled_ser_sept_14_days = ser_sept_14_days.resample('60s').pad()
np_arr = upsampled_ser_sept_14_days.to_numpy()


# We need both a rescaled dataset wrt. the units and a MinMax Scaled
rescaled_ser_sept_14_days = np_arr*(1/(1000*60))

# MinMax
scaled_data = sklearn.preprocessing.minmax_scale(np_arr)

# save data

#np.save("../data/energy_data/price_sept_14_days.npy", rescaled_ser_sept_14_days)
#np.save("../data/energy_data/price_MinMax_sept_14_days.npy", scaled_data)


In [ ]:
print(9.9860e+01)

In [ ]:
plt.plot(rescaled_ser_sept_14_days)
plt.show()

In [ ]:
## Plot Time series:
df_fifteen_min = pd.read_csv("../data/energy_data/Gro_handelspreise_202209010000_202209302359_stunde.csv")
#ser_fifteen_min = df_fifteen_min[df_fifteen_min.columns.to_list()[3]]
df_fifteen_min.columns.to_list()
df_fifteen_min.iloc[:,2]


In [ ]:
ser_fifteen_min = df_fifteen_min['Deutschland/Luxemburg [€/MWh] Interpolierte Auflösungen']